In [ ]:
!git clone https://github.com/amanwalia123/KeyFramesExtraction.git

Cloning into 'KeyFramesExtraction'...
remote: Enumerating objects: 13, done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (13/13), done.


In [ ]:
%cd /content/KeyFramesExtraction

/content/KeyFramesExtraction


In [ ]:
!python scene_div.py /content/BOR08_001_v21.mpg /content/KEYFRAME1/ 500

/usr/bin/python3
Video :/content/BOR08_001_v21.mpg
Frame Directory: /content/KEYFRAME1/
[mpeg1video @ 0x563ac3a7bf80] skipped MB in I-frame at 14 14
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] mb incr damaged
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] ac-tex damaged at 16 9
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] ac-tex damaged at 15 13
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] mb incr damaged
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] slice mismatch
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] mb incr damaged
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] mb incr damaged
[mpeg1video @ 0x563ac3a7bf80] Warning MVs not available
[mpeg1video @ 0x563ac3a7bf80] ac-tex damaged at 15 11
[mpeg1video @

In [ ]:
# -*- coding: utf-8 -*-

import cv2
import operator
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.signal import argrelextrema

print(sys.executable)
#Setting fixed threshold criteria
USE_THRESH = False
#fixed threshold value
THRESH = 0.6
#Setting fixed threshold criteria
USE_TOP_ORDER = False
#Setting local maxima criteria
USE_LOCAL_MAXIMA = True
#Number of top sorted frames
NUM_TOP_FRAMES = 20

#Video path of the source file
videopath = sys.argv[1]
#Directory to store the processed frames
dir = sys.argv[2]
#smoothing window size
len_window = int(sys.argv[3])


def smooth(x, window_len=13, window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    import numpy as np    
    t = np.linspace(-2,2,0.1)
    x = np.sin(t)+np.random.randn(len(t))*0.1
    y = smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string   
    """
    print(len(x), window_len)
    if x.ndim != 1:
        raise( ValueError, "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise(ValueError, "Input vector needs to be bigger than window size.")

    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise (ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s = np.r_[2 * x[0] - x[window_len:1:-1],
              x, 2 * x[-1] - x[-1:-window_len:-1]]
    #print(len(s))

    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = getattr(np, window)(window_len)
    y = np.convolve(w / w.sum(), s, mode='same')
    return y[window_len - 1:-window_len + 1]

#Class to hold information about each frame


class Frame:
    def __init__(self, id, frame, value):
        self.id = id
        self.frame = frame
        self.value = value

    def __lt__(self, other):
        if self.id == other.id:
            return self.id < other.id
        return self.id < other.id

    def __gt__(self, other):
        return other.__lt__(self)

    def __eq__(self, other):
        return self.id == other.id and self.id == other.id

    def __ne__(self, other):
        return not self.__eq__(other)


def rel_change(a, b):
   x = (b - a) / max(a, b)
   print(x)
   return x


print("Video :" + videopath)
print("Frame Directory: " + dir)


cap = cv2.VideoCapture(str(videopath))


curr_frame = None
prev_frame = None

frame_diffs = []
frames = []
ret, frame = cap.read()
i = 1

while(ret):
    luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
    curr_frame = luv
    if curr_frame is not None and prev_frame is not None:
        #logic here
        diff = cv2.absdiff(curr_frame, prev_frame)
        count = np.sum(diff)
        frame_diffs.append(count)
        frame = Frame(i, frame, count)
        frames.append(frame)
    prev_frame = curr_frame
    i = i + 1
    ret, frame = cap.read()
"""
    cv2.imshow('frame',luv)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
"""
cap.release()
#cv2.destroyAllWindows()

if USE_TOP_ORDER:
    # sort the list in descending order
    frames.sort(key=operator.attrgetter("value"), reverse=True)
    for keyframe in frames[:NUM_TOP_FRAMES]:
        name = "frame_" + str(keyframe.id) + ".jpg"
        cv2.imwrite(dir + "/" + name, keyframe.frame)

if USE_THRESH:
    print("Using Threshold")
    for i in range(1, len(frames)):
        if (rel_change(np.float(frames[i - 1].value), np.float(frames[i].value)) >= THRESH):
            #print("prev_frame:"+str(frames[i-1].value)+"  curr_frame:"+str(frames[i].value))
            name = "frame_" + str(frames[i].id) + ".jpg"
            cv2.imwrite(dir + "/" + name, frames[i].frame)


if USE_LOCAL_MAXIMA:
    print("Using Local Maxima")
    diff_array = np.array(frame_diffs)
    sm_diff_array = smooth(diff_array, len_window)
    frame_indexes = np.asarray(argrelextrema(sm_diff_array, np.greater))[0]
    for i in frame_indexes:
        name = "frame_" + str(frames[i - 1].id) + ".jpg"
        print(dir+name)
        cv2.imwrite(dir + name, frames[i - 1].frame)


plt.figure(figsize=(40, 20))
plt.locator_params(numticks=100)
plt.stem(sm_diff_array)
plt.savefig(dir + 'plot.png')


In [23]:
#DOG - 3D SHIFT
import cv2
import numpy as np

def DoG():
    fn = input("Enter image file name and path: ")
    #fn = /content/rose1.jfif
    fn_no_ext = fn.split('.')[0]
    outputFile = fn_no_ext+'DoG.jpg'
    #read the input file
    img = cv2.imread(str(fn))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #run a 5x5 gaussian blur then a 3x3 gaussian blr
    blur5 = cv2.GaussianBlur(img,(5,5),0)
    blur3 = cv2.GaussianBlur(img,(3,3),0)

    #write the results of the previous step to new files
    cv2.imwrite(fn_no_ext+'3x3.jpg', blur3)
    cv2.imwrite(fn_no_ext+'5x5.jpg', blur5)

    DoGim = blur5 - blur3
    img1 = DoGim

    

    img_k = cv2.cornerHarris(img1,2,3,0.04)

    img_k = cv2.dilate(img_k,None)
    print(img_k.max())
# Threshold for an optimal value, it may vary depending on the image.
    #
    img[img_k > (0.01*img_k.max())]=[0,0,255]

    #img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    #gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    #keypoints
    sift = cv2.xfeatures2d_SIFT.create()

    keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(img,None)
    keypoints_3, descriptors_3 = sift.detectAndCompute(img1,None)

    print(DoGim.shape)
    print(len(keypoints_2)) # SIFT
    print(len(keypoints_1)) # 3 D SIFT
    print(len(keypoints_3)) # CORNERS

    
    cv2.imwrite(outputFile, DoGim)
 
def compareImages():
    input1 = raw_input('enter the first image to be compared: ')
    input2 = raw_input('enter the second image to be compared: ')
    outFile = raw_input('enter the filename of the desired output: ')
 
    in1 = cv2.imread(input1)
    in2 = cv2.imread(input2)
 
    output1 = in2 * -1*in1
 
    cv2.imwrite(outFile+'.jpg', output1)
 
print("Welcome to the Difference of Gaussian Image creation utility.")
actionSelection = input("If you would like to compare two images type \"Compare\", if you want to calculate a new set Difference of Gaussian image type \"New\": \n").lower()

if actionSelection == "new":
    DoG()
elif actionSelection == "compare":
    compareImages()
else:
    print("Not a valid selection.")
 

Welcome to the Difference of Gaussian Image creation utility.
If you would like to compare two images type "Compare", if you want to calculate a new set Difference of Gaussian image type "New": 
New
Enter image file name and path: /content/rose1.jfif
0.26578125
(275, 183)
234
1160
1160


In [4]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

     |████████████████████████████████| 25.0MB 182kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 141kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
